In [1]:
spark

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
1,application_1747123548282_0002,pyspark,idle,Link,Link,None,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [2]:
import os
import json

import pyspark
from pyspark.ml import PipelineModel, Pipeline
from pyspark.sql import functions as F

import sparknlp
import sparknlp_jsl
from sparknlp_jsl.annotator import *
from sparknlp_jsl import *
from sparknlp.annotator import *
from sparknlp.base import *
from sparknlp.pretrained import PretrainedPipeline
from sparknlp_jsl.pipeline_tracer import PipelineTracer
from sparknlp_jsl.pipeline_output_parser import PipelineOutputParser
from pyspark.ml import PipelineModel, Pipeline

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [3]:
print("PySpark version:", pyspark.__version__)
print("Spark NLP Version :", sparknlp.version())
print("Spark NLP_JSL Version :", sparknlp_jsl.version())

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

PySpark version: 3.5.0+amzn.0
Spark NLP Version : 6.0.0
Spark NLP_JSL Version : 6.0.0

In [4]:
document_assembler = DocumentAssembler() \
    .setInputCol("text") \
    .setOutputCol("document")

sentence_detector_dl = SentenceDetectorDLModel.load("s3://johnsnowlabs-emr-189352970232/workspace-storage/e-ASLPH2P2XWFN3IABGUI2PKJVS/models/sentence_detector_dl_healthcare_en_3.2.0_3.0_1628678815210/") \
    .setInputCols(["document"]) \
    .setOutputCol("sentence")

tokenizer = Tokenizer() \
    .setInputCols(["sentence"]) \
    .setOutputCol("token")

word_embeddings = WordEmbeddingsModel.load("s3://johnsnowlabs-emr-189352970232/workspace-storage/e-ASLPH2P2XWFN3IABGUI2PKJVS/models/embeddings_clinical_en_2.4.0_2.4_1580237286004/") \
    .setInputCols(["sentence", "token"]) \
    .setOutputCol("embeddings")

clinical_ner = MedicalNerModel.load("s3://johnsnowlabs-emr-189352970232/workspace-storage/e-ASLPH2P2XWFN3IABGUI2PKJVS/models/ner_deid_subentity_augmented_en_3.2.0_2.4_1630671569402/") \
    .setInputCols(["sentence", "token", "embeddings"]) \
    .setOutputCol("ner")

ner_converter = NerConverterInternal() \
    .setInputCols(["sentence", "token", "ner"]) \
    .setOutputCol("ner_chunk")

pretrained_zero_shot_ner = PretrainedZeroShotNER().load("s3://johnsnowlabs-emr-189352970232/workspace-storage/e-ASLPH2P2XWFN3IABGUI2PKJVS/models/zeroshot_ner_deid_subentity_merged_medium_en_5.5.0_3.0_1732701620086/") \
    .setInputCols("sentence", "token") \
    .setOutputCol("zero_shot_ner")

zero_shot_ner_converter = NerConverterInternal() \
    .setInputCols("sentence", "token", "zero_shot_ner") \
    .setOutputCol("zero_shot_ner_chunk")

chunk_merge = ChunkMergeModel() \
    .setInputCols("zero_shot_ner_chunk", "ner") \
    .setOutputCol("deid_merged_chunk")

deidentification = DeIdentification() \
    .setInputCols(["sentence", "token", "deid_merged_chunk"]) \
    .setOutputCol("deidentified") \
    .setMode("obfuscate") \
    .setObfuscateDate(True) \
    .setIgnoreRegex(True) \
    .setSeed(111) \
    .setDays(20)

pipeline = Pipeline(stages=[
    document_assembler, 
    sentence_detector_dl, 
    tokenizer, 
    word_embeddings,
    clinical_ner, 
    ner_converter, 
    pretrained_zero_shot_ner,
    zero_shot_ner_converter, 
    chunk_merge, deidentification
])

empty_data = spark.createDataFrame([[""]]).toDF("text")
model = pipeline.fit(empty_data)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [5]:
text = """Patient Name: John Matthews  
Date of Birth: 03/14/1975  
Visit Date: 04/22/2023  
Age: 48  
Country: United States  
City: San Francisco  
Primary Care Physician: Dr. Elizabeth Warren  

Chief Complaint:  
Mr. Matthews presented to the clinic with persistent chest discomfort and occasional shortness of breath for the past two weeks.

Medical History:  
The patient has a history of hypertension and was previously treated in Toronto, Canada. He reported no known allergies.

Physical Examination:  
Vitals were stable. BP: 130/85, HR: 78 bpm. Lungs were clear on auscultation.

Assessment and Plan:  
Suspected angina. Scheduled for a stress test on 04/25/2023. Continue Lisinopril 10 mg daily and follow up with Dr. Warren in two weeks."""

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [6]:
text_df = spark.createDataFrame([[text]]).toDF("text")
result_df = model.transform(text_df)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
collected = result_df.select("document","sentence", "deidentified").collect()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [8]:
for row in collected:
    deidentifieds = row["deidentified"]
    document = str(row["document"][0].result)
    sentences = row["sentence"]
    sentence_begin = 0
    obfuscated_str = ""
    for index, sent in enumerate(sentences):
        obfuscated_str += document[sentence_begin:sent.begin]
        obfuscated_str += deidentifieds[index].result
        sentence_begin = sent.end + 1
        
    print(obfuscated_str)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Patient Name: Drusilla Fallen  
Date of Birth: 04/03/1975  
Visit Date: 05/12/2023  
Age: 47  
Country: Cayman Islands  
City: Berea  
Primary Care Physician: Dr. Jeanice Branch  

Chief Complaint:  
Mr. Fallen presented to the clinic with persistent chest discomfort and occasional shortness of breath for the past two weeks.

Medical History:  
The patient has a history of hypertension and was previously treated in Sac city, Poland. He reported no known allergies.

Physical Examination:  
Vitals were stable. BP: 130/85, HR: 78 bpm. Lungs were clear on auscultation.

Assessment and Plan:  
Suspected angina. Scheduled for a stress test on 05/15/2023. Continue Lisinopril 10 mg daily and follow up with Dr. Branch in two weeks.